In [15]:
import pandas as pd
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

def OLS_model_predict(region, pred_year_month):
    pred_year = int(str(pred_year_month).split("-")[0])
    month = int(str(pred_year_month).split("-")[-1])

    df_weather = pd.read_csv('static/data/기상데이터 0510 index X.csv')
    df_tmp_region = df_weather[df_weather['조사지역'] == region]
    df_weather_region = df_tmp_region.groupby(['조사지역', '관측일자']).mean()
    df_weather_region = df_weather_region.reset_index(drop=False)

    df_group_mean = df_weather_region.groupby(['연', '월']).mean()
    df_group_mean = df_group_mean.reset_index(drop=False)
    df_month_mean = df_group_mean[df_group_mean['월'] == month]

    for i in df_month_mean.index:
        df_tmp = df_weather_region.loc[(df_weather_region['연'] == df_month_mean.loc[i, '연']) & (df_weather_region['월'] == df_month_mean.loc[i, '월'])]
        temp_max = df_tmp['temp_max'].max()
        temp_min = df_tmp['temp_min'].min()
        df_month_mean.loc[i, 'temp_max'] = temp_max
        df_month_mean.loc[i, 'temp_min'] = temp_min

    pred_list = []
    for value in df_month_mean.columns[2: -3]:
        df_target = df_month_mean[value]
        X_train2 = pd.DataFrame(df_month_mean[['연']], columns = ['연'])
        y_train2 = df_target.values

        # import statsmodels.api as sm
        X_train = sm.add_constant(X_train2)
        model = sm.OLS(y_train2, X_train2).fit()
        pred_value = model.predict(pred_year)

        pred_list.append(float(pred_value))
    
    # print(f'\t\t\t{region}의 {month}월 {pred_year}의 예측값:')
    # print(pred_list)
    return pred_list

In [5]:
df = pd.read_csv('static/data/기상데이터 0510 index X.csv')
region = sorted(df['조사지역'].unique())
region.remove('고산')
region.remove('관악산')
region.remove('대관령')
region.remove('흑산도')
region.remove('문산')
region.remove('추풍령')
len(region)


67

In [ ]:
import numpy as np

for i in range(2081,2090):
    reg_pred_list = []
    for j in range(1,13):
        cnt = 0
        for reg in region:
            pred = OLS_model_predict(reg, f"{i}-{j}")
            reg_pred_list.append(cnt)
            reg_pred_list.append(j)
            reg_pred_list.append(reg)
            for k in pred:
                reg_pred_list.append(k)
    reg_pred_array = np.array(reg_pred_list)
    reg_pred_array = reg_pred_array.reshape(-1,6)
    df2 = pd.DataFrame(reg_pred_array, columns=['KOR_NUM', 'month', 'KOR_NM','temperature', 'temp_max', 'temp_min'])
    df2.to_csv(f"{i}예측.csv", mode='w')
